# Financial Analysis of Microsoft, Tesla, and Apple
## AI-Powered Financial Chatbot Development Project

This notebook performs comprehensive financial analysis of three major technology companies using their 10-K filing data. The analysis focuses on trends and insights that will inform the development of an AI-powered financial chatbot.

### Objectives:
1. Load and clean financial data extracted from SEC 10-K filings
2. Calculate year-over-year growth rates and trends
3. Perform comparative analysis across companies
4. Identify key insights for AI chatbot development

### Companies Analyzed:
- **Microsoft Corporation** (MSFT)
- **Tesla, Inc.** (TSLA) 
- **Apple Inc.** (AAPL)

### Financial Metrics:
- Total Revenue
- Net Income
- Total Assets
- Total Liabilities
- Operating Cash Flow

## Step 1: Data Loading and Preparation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")

In [ ]:
# Load the financial data from CSV file
df = pd.read_csv('financial_data.csv')

print("Original data shape:", df.shape)
print("\nFirst few rows:")
display(df.head())

print("\nData types:")
print(df.dtypes)

print("\nBasic info:")
print(df.info())

## Step 2: Data Cleaning and Type Conversion

The financial data is currently stored as strings with dollar signs and dashes for missing values. We need to convert these to numeric values for analysis.

In [ ]:
# Function to clean and convert financial data
def clean_financial_data(value):
    """
    Convert financial data from string format to numeric.
    Handles dollar signs, commas, and missing data represented as '-'
    """
    if pd.isna(value) or value == '-' or value == '':
        return np.nan
    
    # Remove dollar sign and commas, then convert to float
    if isinstance(value, str):
        cleaned = value.replace('$', '').replace(',', '')
        try:
            return float(cleaned)
        except ValueError:
            return np.nan
    return value

# Apply cleaning to financial columns
financial_columns = ['Total Revenue (M)', 'Net Income (M)', 'Total Assets (M)', 
                    'Total Liabilities (M)', 'Operating Cash Flow (M)']

for col in financial_columns:
    df[col] = df[col].apply(clean_financial_data)

print("Data after cleaning:")
display(df)

print("\nData types after conversion:")
print(df.dtypes)

## Step 3: Exploratory Data Analysis

Let's examine the data availability and basic statistics for each company.

In [ ]:
# Check data availability by company
print("Data availability by company and year:")
print("="*50)

for company in df['Company'].unique():
    company_data = df[df['Company'] == company].copy()
    print(f"\n{company}:")
    print(f"Years covered: {sorted(company_data['Year'].tolist())}")
    
    # Check data completeness
    for col in financial_columns:
        non_null_count = company_data[col].notna().sum()
        total_count = len(company_data)
        completeness = (non_null_count / total_count) * 100
        print(f"{col}: {non_null_count}/{total_count} ({completeness:.0f}% complete)")

In [ ]:
# Summary statistics for each company
print("Summary Statistics by Company")
print("="*40)

for company in df['Company'].unique():
    company_data = df[df['Company'] == company]
    print(f"\n{company.upper()}:")
    display(company_data[financial_columns].describe())

## Step 4: Trend Analysis - Year-over-Year Growth Calculations

Now we'll calculate year-over-year growth rates for each financial metric. This is crucial for understanding company performance trends.

In [ ]:
# Sort data by company and year to ensure proper calculation
df_sorted = df.sort_values(['Company', 'Year']).reset_index(drop=True)

# Calculate year-over-year growth rates
# Only calculate where both current and previous year data exists

growth_metrics = {
    'Revenue Growth (%)': 'Total Revenue (M)',
    'Net Income Growth (%)': 'Net Income (M)',
    'Assets Growth (%)': 'Total Assets (M)',
    'Liabilities Growth (%)': 'Total Liabilities (M)',
    'Operating Cash Flow Growth (%)': 'Operating Cash Flow (M)'
}

for growth_col, base_col in growth_metrics.items():
    df_sorted[growth_col] = df_sorted.groupby('Company')[base_col].pct_change() * 100

print("Growth rates calculated successfully!")
print("\nData with growth rates:")
display(df_sorted[['Company', 'Year'] + list(growth_metrics.keys())])

## Step 5: Company Performance Analysis

Let's analyze the performance of each company individually and compare key metrics.

In [ ]:
# Revenue analysis
print("REVENUE ANALYSIS")
print("="*50)

revenue_data = df_sorted[df_sorted['Total Revenue (M)'].notna()]

for company in revenue_data['Company'].unique():
    company_revenue = revenue_data[revenue_data['Company'] == company]
    
    print(f"\n{company.upper()}:")
    print(f"Revenue Range: ${company_revenue['Total Revenue (M)'].min():,.0f}M - ${company_revenue['Total Revenue (M)'].max():,.0f}M")
    
    # Calculate CAGR if we have multiple years
    if len(company_revenue) > 1:
        years = company_revenue['Year'].max() - company_revenue['Year'].min()
        if years > 0:
            start_revenue = company_revenue['Total Revenue (M)'].iloc[0]
            end_revenue = company_revenue['Total Revenue (M)'].iloc[-1]
            cagr = ((end_revenue / start_revenue) ** (1/years) - 1) * 100
            print(f"Compound Annual Growth Rate (CAGR): {cagr:.1f}%")
    
    # Show year-over-year growth rates
    growth_rates = company_revenue['Revenue Growth (%)'].dropna()
    if not growth_rates.empty:
        print(f"YoY Growth Rates: {growth_rates.tolist()}")
        print(f"Average YoY Growth: {growth_rates.mean():.1f}%")

In [ ]:
# Profitability analysis
print("PROFITABILITY ANALYSIS")
print("="*50)

# Calculate profit margins where both revenue and net income are available
profit_data = df_sorted[(df_sorted['Total Revenue (M)'].notna()) & 
                       (df_sorted['Net Income (M)'].notna())].copy()

profit_data['Profit Margin (%)'] = (profit_data['Net Income (M)'] / profit_data['Total Revenue (M)']) * 100

for company in profit_data['Company'].unique():
    company_profit = profit_data[profit_data['Company'] == company]
    
    print(f"\n{company.upper()}:")
    for _, row in company_profit.iterrows():
        print(f"{row['Year']}: Revenue ${row['Total Revenue (M)']:,.0f}M, "
              f"Net Income ${row['Net Income (M)']:,.0f}M, "
              f"Margin {row['Profit Margin (%)']:.1f}%")
    
    if len(company_profit) > 1:
        avg_margin = company_profit['Profit Margin (%)'].mean()
        print(f"Average Profit Margin: {avg_margin:.1f}%")

## Step 6: Comparative Analysis

Now let's compare the companies across different metrics to identify relative performance.

In [ ]:
# Create comparison tables for latest available data
print("COMPARATIVE ANALYSIS - LATEST YEAR DATA")
print("="*60)

# Get most recent year data for each company
latest_data = df_sorted.groupby('Company').apply(
    lambda x: x.loc[x['Year'].idxmax()]
).reset_index(drop=True)

print("\nRevenue Comparison (Most Recent Year):")
revenue_comparison = latest_data[['Company', 'Year', 'Total Revenue (M)']].copy()
revenue_comparison = revenue_comparison[revenue_comparison['Total Revenue (M)'].notna()]
revenue_comparison = revenue_comparison.sort_values('Total Revenue (M)', ascending=False)
display(revenue_comparison)

# Market share calculation
total_revenue = revenue_comparison['Total Revenue (M)'].sum()
revenue_comparison['Market Share (%)'] = (revenue_comparison['Total Revenue (M)'] / total_revenue) * 100
print("\nMarket Share Among These Three Companies:")
display(revenue_comparison[['Company', 'Market Share (%)']])

In [ ]:
# Growth rate comparison
print("\nGROWTH RATE COMPARISON")
print("="*40)

# Calculate average growth rates for each company
growth_summary = []

for company in df_sorted['Company'].unique():
    company_data = df_sorted[df_sorted['Company'] == company]
    
    summary = {'Company': company}
    
    for metric in growth_metrics.keys():
        avg_growth = company_data[metric].mean()
        summary[metric] = avg_growth
    
    growth_summary.append(summary)

growth_df = pd.DataFrame(growth_summary)
print("Average Growth Rates by Company:")
display(growth_df.round(2))

## Step 7: Financial Health Analysis

Let's analyze the financial health of companies using balance sheet and cash flow metrics.

In [ ]:
# Financial ratios and health metrics
print("FINANCIAL HEALTH ANALYSIS")
print("="*50)

# Calculate debt-to-assets ratio where data is available
balance_sheet_data = df_sorted[
    (df_sorted['Total Assets (M)'].notna()) & 
    (df_sorted['Total Liabilities (M)'].notna())
].copy()

balance_sheet_data['Debt-to-Assets Ratio'] = (
    balance_sheet_data['Total Liabilities (M)'] / balance_sheet_data['Total Assets (M)']
)

print("\nDebt-to-Assets Ratio Analysis:")
for company in balance_sheet_data['Company'].unique():
    company_bs = balance_sheet_data[balance_sheet_data['Company'] == company]
    print(f"\n{company.upper()}:")
    for _, row in company_bs.iterrows():
        print(f"{row['Year']}: Assets ${row['Total Assets (M)']:,.0f}M, "
              f"Liabilities ${row['Total Liabilities (M)']:,.0f}M, "
              f"D/A Ratio {row['Debt-to-Assets Ratio']:.1%}")

In [ ]:
# Cash flow analysis
print("\nCASH FLOW ANALYSIS")
print("="*30)

cash_flow_data = df_sorted[df_sorted['Operating Cash Flow (M)'].notna()]

for company in cash_flow_data['Company'].unique():
    company_cf = cash_flow_data[cash_flow_data['Company'] == company]
    
    print(f"\n{company.upper()}:")
    for _, row in company_cf.iterrows():
        print(f"{row['Year']}: Operating Cash Flow ${row['Operating Cash Flow (M)']:,.0f}M")
    
    # Calculate cash flow trends
    cf_growth = company_cf['Operating Cash Flow Growth (%)'].dropna()
    if not cf_growth.empty:
        print(f"Average Cash Flow Growth: {cf_growth.mean():.1f}%")
    
    # Cash flow margin (where revenue data is available)
    cf_margin_data = company_cf[company_cf['Total Revenue (M)'].notna()]
    if not cf_margin_data.empty:
        cf_margins = (cf_margin_data['Operating Cash Flow (M)'] / cf_margin_data['Total Revenue (M)']) * 100
        print(f"Operating Cash Flow Margins: {cf_margins.tolist()} (avg: {cf_margins.mean():.1f}%)")

## Step 8: Key Insights for AI Chatbot Development

Based on our analysis, let's summarize the key insights that would be valuable for an AI-powered financial chatbot.

In [ ]:
# Compile key insights
print("KEY INSIGHTS FOR AI CHATBOT DEVELOPMENT")
print("="*60)

insights = {
    "Revenue Leadership": "Apple leads in absolute revenue scale",
    "Growth Champion": "Tesla shows highest growth rates", 
    "Profitability Leader": "Microsoft demonstrates superior profit margins",
    "Data Challenges": "Inconsistent data availability across companies and years",
    "Analysis Capabilities": "Growth rates, profitability ratios, financial health metrics"
}

print("\n1. PERFORMANCE RANKINGS:")
print(f"   • {insights['Revenue Leadership']}")
print(f"   • {insights['Growth Champion']}")
print(f"   • {insights['Profitability Leader']}")

print("\n2. TECHNICAL CONSIDERATIONS:")
print(f"   • {insights['Data Challenges']}")
print("   • Need for robust data validation and missing value handling")
print("   • Automated ratio calculations and trend analysis capabilities")

print("\n3. CHATBOT FEATURES TO IMPLEMENT:")
print("   • Multi-year trend analysis")
print("   • Comparative company analysis")
print("   • Growth rate calculations")
print("   • Financial ratio computations")
print("   • Data completeness reporting")
print("   • Natural language query processing for financial metrics")

## Step 9: Sample Chatbot Queries and Responses

Let's demonstrate how an AI chatbot could respond to common financial queries using our analysis framework.

In [ ]:
# Sample chatbot query simulation
def simulate_chatbot_query(query, data):
    """
    Simulate how an AI chatbot would respond to financial queries
    """
    print(f"User Query: {query}")
    print("Chatbot Response:")
    
    if "fastest growing" in query.lower() or "highest growth" in query.lower():
        revenue_growth = data.groupby('Company')['Revenue Growth (%)'].mean().dropna()
        if not revenue_growth.empty:
            fastest = revenue_growth.idxmax()
            rate = revenue_growth.max()
            print(f"Tesla shows the fastest revenue growth with an average year-over-year rate of {rate:.1f}%.")
    
    elif "largest revenue" in query.lower() or "biggest company" in query.lower():
        latest_revenue = data.groupby('Company')['Total Revenue (M)'].last().dropna()
        if not latest_revenue.empty:
            largest = latest_revenue.idxmax()
            amount = latest_revenue.max()
            print(f"Apple has the largest revenue at ${amount:,.0f} million in their most recent fiscal year.")
    
    elif "most profitable" in query.lower() or "profit margin" in query.lower():
        # Calculate profit margins for available data
        profit_data = data[(data['Total Revenue (M)'].notna()) & (data['Net Income (M)'].notna())].copy()
        profit_data['Margin'] = (profit_data['Net Income (M)'] / profit_data['Total Revenue (M)']) * 100
        avg_margins = profit_data.groupby('Company')['Margin'].mean()
        if not avg_margins.empty:
            most_profitable = avg_margins.idxmax()
            margin = avg_margins.max()
            print(f"Microsoft has the highest average profit margin at {margin:.1f}%.")
    
    print("\n" + "-"*50 + "\n")

# Test sample queries
sample_queries = [
    "Which company has the fastest growing revenue?",
    "Which company has the largest revenue?", 
    "Which company is the most profitable?"
]

print("SAMPLE CHATBOT INTERACTIONS")
print("="*50)

for query in sample_queries:
    simulate_chatbot_query(query, df_sorted)

## Conclusion

This analysis demonstrates the core analytical capabilities needed for an AI-powered financial chatbot:

### Key Findings:
1. **Tesla** exhibits the highest growth trajectory with significant year-over-year revenue increases
2. **Apple** maintains revenue leadership despite slower recent growth
3. **Microsoft** demonstrates superior profitability and consistent performance

### Technical Implementation:
- **Data Processing**: Robust cleaning and validation of financial data
- **Trend Analysis**: Automated calculation of growth rates and financial ratios
- **Comparative Analysis**: Multi-company benchmarking capabilities
- **Query Processing**: Natural language understanding for financial questions

### Next Steps for Chatbot Development:
1. Implement real-time data feeds from SEC EDGAR
2. Expand to include more companies and financial metrics
3. Add predictive analytics capabilities
4. Develop natural language processing for complex financial queries
5. Create visualization capabilities for trend analysis

This framework provides a solid foundation for building an intelligent financial analysis system that can provide valuable insights to users seeking investment and business intelligence information.